In [33]:
import os
import random
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from glob import glob
from PIL import Image, ImageFile
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import confusion_matrix, roc_auc_score, precision_recall_fscore_support
import matplotlib.pyplot as plt
import pandas as pd
from itertools import product
from tqdm import tqdm
from torch.optim import Adam
from torchvision import datasets
from adopt import ADOPT
from torch.optim import Adam
"""
@inproceedings{taniguchi2024adopt,
 author={Taniguchi, Shohei and Harada, Keno and Minegishi, Gouki and Oshima, Yuta and Jeong, Seong Cheol and Nagahara, Go and Iiyama, Tomoshi and Suzuki, Masahiro and Iwasawa, Yusuke and Matsuo, Yutaka},
 booktitle = {Advances in Neural Information Processing Systems},
 title = {ADOPT: Modified Adam Can Converge with Any β2 with the Optimal Rate},
 year = {2024}
}
"""

'\n@inproceedings{taniguchi2024adopt,\n author={Taniguchi, Shohei and Harada, Keno and Minegishi, Gouki and Oshima, Yuta and Jeong, Seong Cheol and Nagahara, Go and Iiyama, Tomoshi and Suzuki, Masahiro and Iwasawa, Yusuke and Matsuo, Yutaka},\n booktitle = {Advances in Neural Information Processing Systems},\n title = {ADOPT: Modified Adam Can Converge with Any β2 with the Optimal Rate},\n year = {2024}\n}\n'

In [34]:
# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [35]:
# Constants
ROOT_DIR = "Datasets/corrected_wildfires_dataset"
IMAGE_SIZE = (224, 224)
MEAN = [0.485, 0.456, 0.406]  # Normalization mean
STD = [0.229, 0.224, 0.225]   # Normalization std
NUM_FOLDS = 10
NUM_EPOCHS = 10
NUM_CLASSES = 2

# Hyperparameter ranges for tuning
learning_rates = [0.01, 0.001, 0.0001]
dropout_rates = [0.3, 0.5]
batch_sizes = [16, 32]
num_conv_layers_list = [2, 3]
num_filters_list = [[32, 64], [64, 128]]
fc_units_list = [128, 256]

# Suppress PIL DecompressionBombWarning and handle large images
Image.MAX_IMAGE_PIXELS = None
ImageFile.LOAD_TRUNCATED_IMAGES = True  # Handle truncated images

In [36]:
# Data augmentation and normalization
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((256, 256)),  # Resize images to a reasonable size
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),  # Small rotation
        transforms.RandomResizedCrop(IMAGE_SIZE[0], scale=(0.9, 1.0)),  # Small random zoom in
        transforms.ColorJitter(contrast=0.1),  # Small random contrast
        transforms.ToTensor(),
        transforms.Normalize(MEAN, STD)
    ]),
    'val': transforms.Compose([
        transforms.Resize((256, 256)),  # Resize images to a reasonable size
        transforms.CenterCrop(IMAGE_SIZE),
        transforms.ToTensor(),
        transforms.Normalize(MEAN, STD)
    ]),
}

In [37]:
class Dataset(torch.utils.data.Dataset):
    """
    Custom Dataset class for loading images and labels.
    """
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform
    
    def __len__(self):
        """Return the total number of samples."""
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        """Generate one sample of data."""
        image_path = self.image_paths[idx]
        label = self.labels[idx]
        try:
            image = Image.open(image_path).convert("RGB")
        except Exception as e:
            # If image is corrupted, replace it with a black image
            print(f"Warning: Unable to open image {image_path}. Error: {e}")
            image = Image.new('RGB', IMAGE_SIZE)
        if self.transform:
            image = self.transform(image)
        return image, label

In [38]:
# Prepare dataset and labels
image_paths = glob(f"{ROOT_DIR}/*/*.jpg")
class_labels = {"fire": 0, "nofire": 1}

# Filter image paths and extract labels
filtered_image_paths = []
labels = []
for p in image_paths:
    label_name = os.path.basename(os.path.dirname(p))
    if label_name in class_labels:
        filtered_image_paths.append(p)
        labels.append(class_labels[label_name])

# Convert to numpy arrays
image_paths = np.array(filtered_image_paths)
labels = np.array(labels)

# Check class distribution before balancing
unique, counts = np.unique(labels, return_counts=True)
class_counts = dict(zip(unique, counts))
print("Class distribution before balancing:", class_counts)

Class distribution before balancing: {0: 469, 1: 990}


In [39]:
# Balance the classes by undersampling the majority class
min_count = min(class_counts.values())

balanced_image_paths = []
balanced_labels = []

for class_label in np.unique(labels):
    class_indices = np.where(labels == class_label)[0]
    sampled_indices = np.random.choice(class_indices, min_count, replace=False)
    balanced_image_paths.extend(image_paths[sampled_indices])
    balanced_labels.extend([class_label] * min_count)

# Convert to numpy arrays
balanced_image_paths = np.array(balanced_image_paths)
balanced_labels = np.array(balanced_labels)

# Shuffle the balanced dataset
indices = np.arange(len(balanced_image_paths))
np.random.shuffle(indices)
balanced_image_paths = balanced_image_paths[indices]
balanced_labels = balanced_labels[indices]

# Update image_paths and labels to the balanced dataset
image_paths = balanced_image_paths
labels = balanced_labels

# Check class distribution after balancing
unique, counts = np.unique(labels, return_counts=True)
class_counts = dict(zip(unique, counts))
print("Class distribution after balancing:", class_counts)

Class distribution after balancing: {0: 469, 1: 469}


In [40]:
# Split data into training and validation sets
train_paths, val_paths, train_labels, val_labels = train_test_split(
    image_paths, labels, test_size=0.2, stratify=labels, random_state=42
)

# Create a list of hyperparameter combinations
hyperparameter_list = []

for lr, dr, bs, ncl, nf, fc in product(learning_rates, dropout_rates, batch_sizes, num_conv_layers_list, num_filters_list, fc_units_list):
    if len(nf) >= ncl:
        architecture = {
            'name': f"Model_lr{lr}_dr{dr}_bs{bs}_ncl{ncl}_nf{'_'.join(map(str, nf[:ncl]))}_fc{fc}",
            'num_conv_layers': ncl,
            'num_filters': nf[:ncl],
            'kernel_sizes': [3]*ncl,
            'fc_units': fc,
            'dropout_rate': dr,
            'learning_rate': lr,
            'batch_size': bs,
            'optimizer': 'adam'
        }
        hyperparameter_list.append(architecture)

In [41]:
class CNNModel(nn.Module):
    """
    Convolutional Neural Network Model.
    """
    def __init__(self, architecture):
        super(CNNModel, self).__init__()
        layers = []
        in_channels = 3
        # Build convolutional layers
        for i in range(architecture['num_conv_layers']):
            out_channels = architecture['num_filters'][i]
            layers.append(nn.Conv2d(in_channels, out_channels, kernel_size=architecture['kernel_sizes'][i], padding=1))
            layers.append(nn.BatchNorm2d(out_channels))
            layers.append(nn.ReLU(inplace=True))
            layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
            in_channels = out_channels
        self.features = nn.Sequential(*layers)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        # Fully connected layers
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_channels, architecture['fc_units']),
            nn.ReLU(inplace=True),
            nn.Dropout(architecture['dropout_rate']),
            nn.Linear(architecture['fc_units'], NUM_CLASSES)
        )
    
    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = self.fc(x)
        return x

In [ ]:
# Initialize results list
results = []

# Hyperparameter tuning loop
for architecture in hyperparameter_list:
    print(f"\nEvaluating Model:")
    print(f"Model Name: {architecture['name']}")
    print(f"Learning Rate: {architecture['learning_rate']}")
    print(f"Dropout Rate: {architecture['dropout_rate']}")
    print(f"Batch Size: {architecture['batch_size']}")
    # Create datasets and dataloaders
    train_dataset = Dataset(train_paths, train_labels, transform=data_transforms['train'])
    val_dataset = Dataset(val_paths, val_labels, transform=data_transforms['val'])
    
    train_loader = DataLoader(train_dataset, batch_size=architecture['batch_size'], shuffle=True, num_workers=4)
    val_loader = DataLoader(val_dataset, batch_size=architecture['batch_size'], shuffle=False, num_workers=4)
    
    # Create model
    model = CNNModel(architecture).to(device)
    
    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=architecture['learning_rate']) #, decoupled=True)
    
    # Implement early stopping
    best_val_loss = float('inf')
    patience = 3
    trigger_times = 0
    best_model_wts = None
    
    train_acc_history = []
    val_acc_history = []
    
    # Training loop
    for epoch in range(NUM_EPOCHS):
        model.train()
        running_loss = 0.0
        running_corrects = 0
        
        # Use tqdm for progress bar
        for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS} Training"):
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            # Zero the parameter gradients
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)
            
            # Check for NaN loss
            if torch.isnan(loss):
                print("Loss is NaN, skipping batch")
                continue
            
            # Backward pass and optimization
            loss.backward()
            optimizer.step()
            
            # Statistics
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
        
        epoch_loss = running_loss / len(train_dataset)
        epoch_acc = running_corrects.double() / len(train_dataset)
        train_acc_history.append(epoch_acc.item())
        
        # Validation
        model.eval()
        val_running_loss = 0.0
        val_running_corrects = 0
        val_labels_list = []
        val_preds_list = []
        val_probs_list = []
        
        with torch.no_grad():
            for inputs, labels in tqdm(val_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS} Validation"):
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)
                
                val_running_loss += loss.item() * inputs.size(0)
                val_running_corrects += torch.sum(preds == labels.data)
                
                val_labels_list.extend(labels.cpu().numpy())
                val_preds_list.extend(preds.cpu().numpy())
                probs = nn.functional.softmax(outputs, dim=1)
                val_probs_list.extend(probs[:, 1].cpu().numpy())
        
        val_epoch_loss = val_running_loss / len(val_dataset)
        val_epoch_acc = val_running_corrects.double() / len(val_dataset)
        val_acc_history.append(val_epoch_acc.item())
        
        print(f"Epoch {epoch+1}/{NUM_EPOCHS}, Train Acc: {epoch_acc:.4f}, Val Acc: {val_epoch_acc:.4f}")
        
        # Early stopping
        if val_epoch_loss < best_val_loss:
            best_val_loss = val_epoch_loss
            best_model_wts = model.state_dict()
            trigger_times = 0
        else:
            trigger_times += 1
            if trigger_times >= patience:
                print("Early stopping")
                break
    
    # Load best model weights
    if best_model_wts:
        model.load_state_dict(best_model_wts)
    
    # Evaluation
    y_true = np.array(val_labels_list)
    y_pred = np.array(val_preds_list)
    val_probs_list = np.array(val_probs_list)
    
    # Handle NaN values in probabilities
    if np.isnan(val_probs_list).any():
        print("NaN values found in validation probabilities, skipping this model.")
        continue
    
    # Metrics
    cm = confusion_matrix(y_true, y_pred)
    precision, recall, f1_score, _ = precision_recall_fscore_support(y_true, y_pred, average='binary', zero_division=0)
    # For ROC AUC, we need probabilities
    try:
        roc_auc = roc_auc_score(y_true, val_probs_list)
    except ValueError as e:
        print(f"ROC AUC Error: {e}, setting roc_auc to 0.0")
        roc_auc = 0.0
    tn, fp, fn, tp = cm.ravel()
    sensitivity = recall  # Same as recall
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0.0
    
    # Store results
    results.append({
        'name': architecture['name'],
        'architecture': architecture,
        'val_accuracy': val_epoch_acc.item(),
        'precision': precision,
        'recall': recall,
        'f1_score': f1_score,
        'roc_auc': roc_auc,
        'tp': tp,
        'fp': fp,
        'tn': tn,
        'fn': fn,
        'sensitivity': sensitivity,
        'specificity': specificity,
        'train_acc_history': train_acc_history,
        'val_acc_history': val_acc_history
    })
    
    # Free up memory
    del model
    torch.cuda.empty_cache()

# Create a DataFrame with the results
results_df = pd.DataFrame(results)


Evaluating Model:
Model Name: Model_lr0.01_dr0.3_bs16_ncl2_nf32_64_fc128
Learning Rate: 0.01
Dropout Rate: 0.3
Batch Size: 16


Epoch 1/10 Training:   0%|          | 0/47 [00:00<?, ?it/s]

Epoch 1/10 Validation: 100%|██████████| 12/12 [00:34<00:00,  2.89s/it]


Epoch 1/10, Train Acc: 0.6760, Val Acc: 0.7234


Epoch 2/10 Validation: 100%|██████████| 12/12 [00:34<00:00,  2.91s/it]


...

Epoch 3/10 Validation: 100%|██████████| 6/6 [00:38<00:00,  6.42s/it]


Epoch 3/10, Train Acc: 0.7200, Val Acc: 0.7766


Epoch 4/10 Validation: 100%|██████████| 6/6 [00:37<00:00,  6.23s/it]


Epoch 4/10, Train Acc: 0.7293, Val Acc: 0.7872


Epoch 5/10 Training:  67%|██████▋   | 16/24 [01:48<00:49,  6.18s/it]

In [ ]:
# Display the table with all architectures and their metrics
print("\nResults Summary:")
print(results_df[['name', 'val_accuracy', 'precision', 'recall', 'f1_score', 'roc_auc', 'sensitivity', 'specificity']])

In [ ]:
# Find the best performing model based on validation accuracy
best_model_idx = results_df['val_accuracy'].idxmax()
best_model_info = results_df.loc[best_model_idx]
print("\nBest Performing Model:")
print(best_model_info['name'])
print(best_model_info['architecture'])

In [ ]:
# Stratified K-Fold Cross Validation
skf = StratifiedKFold(n_splits=NUM_FOLDS, shuffle=True, random_state=42)

# Conduct K-Fold validation on the best architecture
best_architecture = best_model_info['architecture']
print(f"\nConducting K-Fold Validation on Best Architecture: {best_architecture['name']}")

fold_metrics = []

for fold, (train_idx, val_idx) in enumerate(skf.split(image_paths, labels)):
    print(f"Fold {fold+1}/{NUM_FOLDS}")
    # Create datasets and dataloaders
    train_paths_fold, val_paths_fold = image_paths[train_idx], image_paths[val_idx]
    train_labels_fold, val_labels_fold = labels[train_idx], labels[val_idx]
    
    train_dataset = Dataset(train_paths_fold, train_labels_fold, transform=data_transforms['train'])
    val_dataset = Dataset(val_paths_fold, val_labels_fold, transform=data_transforms['val'])
    
    train_loader = DataLoader(train_dataset, batch_size=best_architecture['batch_size'], shuffle=True, num_workers=4)
    val_loader = DataLoader(val_dataset, batch_size=best_architecture['batch_size'], shuffle=False, num_workers=4)
    
    # Create model
    model = CNNModel(best_architecture).to(device)
    
    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=best_architecture['learning_rate'])
    
    # Implement early stopping
    best_val_loss = float('inf')
    patience = 3
    trigger_times = 0
    best_model_wts = None
    
    train_acc_history = []
    val_acc_history = []
    
    # Training loop
    for epoch in range(NUM_EPOCHS):
        model.train()
        running_loss = 0.0
        running_corrects = 0
        
        # Use tqdm for progress bar
        for inputs, labels in tqdm(train_loader, desc=f"Fold {fold+1} Epoch {epoch+1}/{NUM_EPOCHS} Training"):
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            # Zero the parameter gradients
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)
            
            # Check for NaN loss
            if torch.isnan(loss):
                print("Loss is NaN, skipping batch")
                continue
            
            # Backward pass and optimization
            loss.backward()
            optimizer.step()
            
            # Statistics
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
        
        epoch_loss = running_loss / len(train_dataset)
        epoch_acc = running_corrects.double() / len(train_dataset)
        train_acc_history.append(epoch_acc.item())
        
        # Validation
        model.eval()
        val_running_loss = 0.0
        val_running_corrects = 0
        val_labels_list = []
        val_preds_list = []
        val_probs_list = []
        
        with torch.no_grad():
            for inputs, labels in tqdm(val_loader, desc=f"Fold {fold+1} Epoch {epoch+1}/{NUM_EPOCHS} Validation"):
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)
                
                val_running_loss += loss.item() * inputs.size(0)
                val_running_corrects += torch.sum(preds == labels.data)
                
                val_labels_list.extend(labels.cpu().numpy())
                val_preds_list.extend(preds.cpu().numpy())
                probs = nn.functional.softmax(outputs, dim=1)
                val_probs_list.extend(probs[:, 1].cpu().numpy())
        
        val_epoch_loss = val_running_loss / len(val_dataset)
        val_epoch_acc = val_running_corrects.double() / len(val_dataset)
        val_acc_history.append(val_epoch_acc.item())
        
        print(f"Fold {fold+1}, Epoch {epoch+1}/{NUM_EPOCHS}, Train Acc: {epoch_acc:.4f}, Val Acc: {val_epoch_acc:.4f}")
        
        # Early stopping
        if val_epoch_loss < best_val_loss:
            best_val_loss = val_epoch_loss
            best_model_wts = model.state_dict()
            trigger_times = 0
        else:
            trigger_times += 1
            if trigger_times >= patience:
                print("Early stopping")
                break
    
    # Load best model weights
    if best_model_wts:
        model.load_state_dict(best_model_wts)
    
    # Evaluation
    y_true = np.array(val_labels_list)
    y_pred = np.array(val_preds_list)
    val_probs_list = np.array(val_probs_list)
    
    # Handle NaN values in probabilities
    if np.isnan(val_probs_list).any():
        print("NaN values found in validation probabilities, skipping this fold.")
        continue
    
    # Metrics
    cm = confusion_matrix(y_true, y_pred)
    precision, recall, f1_score, _ = precision_recall_fscore_support(y_true, y_pred, average='binary', zero_division=0)
    # For ROC AUC, we need probabilities
    try:
        roc_auc = roc_auc_score(y_true, val_probs_list)
    except ValueError as e:
        print(f"ROC AUC Error: {e}, setting roc_auc to 0.0")
        roc_auc = 0.0
    tn, fp, fn, tp = cm.ravel()
    sensitivity = recall
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0.0
    
    fold_metrics.append({
        'fold': fold+1,
        'val_accuracy': val_epoch_acc.item(),
        'precision': precision,
        'recall': recall,
        'f1_score': f1_score,
        'roc_auc': roc_auc,
        'tp': tp,
        'fp': fp,
        'tn': tn,
        'fn': fn,
        'sensitivity': sensitivity,
        'specificity': specificity
    })
    
    # Free up memory
    del model
    torch.cuda.empty_cache()

# Average metrics over folds
kfold_results_df = pd.DataFrame(fold_metrics)
avg_metrics = kfold_results_df.mean()
print("\nK-Fold Cross-Validation Results:")
print(kfold_results_df)
print("\nAverage Metrics over all folds:")
print(avg_metrics)

In [ ]:
# Use the entire balanced dataset for training
final_architecture = best_architecture.copy()
train_dataset = Dataset(image_paths, labels, transform=data_transforms['train'])
train_loader = DataLoader(train_dataset, batch_size=final_architecture['batch_size'], shuffle=True, num_workers=4)

# Create model
model = CNNModel(final_architecture).to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=final_architecture['learning_rate'], decoupled=True)

# Implement early stopping
best_loss = float('inf')
patience = 3
trigger_times = 0
best_model_wts = None

train_acc_history = []

# Training loop
for epoch in range(NUM_EPOCHS):
    model.train()
    running_loss = 0.0
    running_corrects = 0
    
    # Use tqdm for progress bar
    for inputs, labels in tqdm(train_loader, desc=f"Final Training Epoch {epoch+1}/{NUM_EPOCHS}"):
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
        
        # Check for NaN loss
        if torch.isnan(loss):
            print("Loss is NaN, skipping batch")
            continue
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        # Statistics
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
    
    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = running_corrects.double() / len(train_dataset)
    train_acc_history.append(epoch_acc.item())
    
    print(f"Epoch {epoch+1}/{NUM_EPOCHS}, Train Loss: {epoch_loss:.4f}, Train Acc: {epoch_acc:.4f}")
    
    # Early stopping
    if epoch_loss < best_loss:
        best_loss = epoch_loss
        best_model_wts = model.state_dict()
        trigger_times = 0
    else:
        trigger_times += 1
        if trigger_times >= patience:
            print("Early stopping")
            break

# Load best model weights
if best_model_wts:
    model.load_state_dict(best_model_wts)

# Save the final model
torch.save(model.state_dict(), 'final_model.pth')

In [ ]:
# Plot Training Accuracy over Time
plt.figure()
plt.plot(train_acc_history, label='Training Accuracy')
plt.title('Training Accuracy Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
# Final Evaluation
model.eval()
all_preds = []
all_labels = []
all_probs = []
with torch.no_grad():
    for inputs, labels in tqdm(train_loader, desc="Final Evaluation"):
        inputs = inputs.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        probs = nn.functional.softmax(outputs, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.numpy())
        all_probs.extend(probs[:, 1].cpu().numpy())

y_true = np.array(all_labels)
y_pred = np.array(all_preds)
all_probs = np.array(all_probs)

# Handle NaN values in probabilities
if np.isnan(all_probs).any():
    print("NaN values found in probabilities, setting ROC AUC to 0.0")
    roc_auc = 0.0
else:
    roc_auc = roc_auc_score(y_true, all_probs)

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred)
plt.figure()
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix on Full Dataset')
plt.colorbar()
tick_marks = np.arange(NUM_CLASSES)
classes = ['Fire', 'No Fire']
plt.xticks(tick_marks, classes, rotation=45)
plt.yticks(tick_marks, classes)
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

In [ ]:
# Final Metrics
precision, recall, f1_score, _ = precision_recall_fscore_support(y_true, y_pred, average='binary', zero_division=0)
tn, fp, fn, tp = cm.ravel()
sensitivity = recall
specificity = tn / (tn + fp) if (tn + fp) > 0 else 0.0

print("\nFinal Evaluation Metrics on Full Dataset:")
print(f"Accuracy: {train_acc_history[-1]:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1_score:.4f}")
print(f"ROC AUC: {roc_auc:.4f}")
print(f"Sensitivity: {sensitivity:.4f}")
print(f"Specificity: {specificity:.4f}")
print(f"TP: {tp}, FP: {fp}, TN: {tn}, FN: {fn}")